FBL power curve viewer
=====

This colab is just for viewing the json configs and plots that were exported by an FBL run to the shared drive folder, [Makani_FBL/PowerCurves](https://drive.google.com/corp/drive/folders/1c0aHBj8S0jukpC3h44n_t2tTV7XEFD-L)

Note, these json configs do not store the full information neeeded to recreate a run, as they can only store the json serializable parameters and not, for example, methods of the kite config used. Please reference the original notebook used to do the run and any notes in the kite description.

## Instructions

This is just set up as an external colab that you don't need a local runtime for. Make sure the address bar is pointed toward "colab.sandbox.google.com/...", then connect to a hosted runtime, run the import steps and authenticate it, select which folder has the curve you want to run and which basename within that folder (the cells that have 'User-input' in the title), and then run the remaining fields.

To lock a view, probably best to just make a copy of this notebook with the target power curve loaded and run...

>[FBL power curve viewer](#scrollTo=ubff5BT3Yev6)

>>[Instructions](#scrollTo=ubff5BT3Yev6)

>[Setup and import](#scrollTo=rEdjTM9yZlnS)

>[View available options and select power curve to view](#scrollTo=io1iRzV1bbRI)

>[View the selected power curve](#scrollTo=2X9wwApidGWX)



# Setup and import 

In [ ]:
#@title Imports

from collections import OrderedDict
import copy
import json
import os
from pprint import pprint as pp
from IPython import display as dp

import cv2
from google.colab.patches import cv2_imshow

# from matplotlib import pyplot as plt
# from matplotlib import image as mpimg
# from IPython.display import HTML
# from IPython.display import Image

from google.colab import drive


In [ ]:
#@title Authorize drive and mount
drive.mount('/gdrive')


In [ ]:
#@title Set base folder paths and other settings

plot_order = [
              'power_curve.png',
              'paths_locations.png',
              'path_loop_data.png',
              'loop_data.png',
              'angular_rates.png',
              'pose_data.png',
              'paths.png',
              'values.png',
              'constraints.png',
]

power_curve_parent_path = '/gdrive/Shared drives/Makani_FBL/PowerCurves'

print('Using the parent path:')
print(power_curve_parent_path)


# Extract descriptions of config variables from doc string
# kpc_doc = inspect.getdoc(makani_FBL.KitePowerCurve)
# exec('if 1:\n'+source[source.find(def_strs[0]):source.find(def_strs[1])])
# config = kpc_doc[kpc_doc.find('config:'):kpc_doc.find('Kwargs:')]
config = "      <aero_model>:\n        An aero model for the kite (no tether) must be provided.\n        options are:\n          aero_coeff_from_alpha_beta:\n            as above, but includes aero side lift coefficient, cY\n          body_coeff_from_alpha_beta:\n            as above, but body coefficients including cy.\n            note that cY is aero coefficient, and cy is body.\n      eta_shaft_to_pad:\n        a function of (power_shaft) that returns a total efficiency for\n        the system from shaft to padmount\n      m_kite:\n        mass of the kite in kg\n      m_tether:\n        mass of the tether in kg\n      power_shaft_max:\n        maximum net (all motors) shaft power limit, in watts\n      s:\n        kite reference wing area, in m^2\n      l_tether:\n        length of the tether from GS attach point to the wing\n        (including bridle length)\n      shaft_power_from_drag_power:\n        a function of (rho, v_a, rotor_force) that returns a dictionary\n        a negative force is generation\n\n        Required items:\n          power_shaft:\n            shaft power, in W\n      alpha_min:\n        Minimum alpha, applied as a constraint.\n      alpha_max:\n        Maximum alpha, applied as a constraint.\n      cL_max:\n        Maximum coefficient of lift, applied as a constraint.\n      cL_offset:\n        Additional aero coefficient offset to be applied to the kite\n      cY_offset:\n        Additional lateral lift coefficient offset to be applied to the kite\n      cD_offset:\n        Additional drag coefficient offset to be applied to the kite\n      cD_eff_tether:\n        Effective drag coefficient of the tether at the kite, referenced to\n        wing area.\n      tension_max:\n        Maximum operating tension, in Newtons. Applied as a constraint.\n      tension_min:\n        Min operating tension, in Newtons. Applied as a constraint.\n      incl_max:\n        Maximum inclination for any individual pose. Applied as a constraint.\n      h_min:\n        Minimum height AGL for any individual pose. Applied as a constraint.\n      roll_min:\n        Minimum roll angle, tether to kite.\n        Positive roll is in to nominal clockwise looking downwind circle.\n        Ie: rolling to kite's left.\n      roll_max:\n        Maximum roll angle, tether to kite.\n        See roll_min for details.\n      v_a_min:\n        Minimum allowable airspeed for the kite, in m/s.\n      v_a_max:\n        Maximum allowable airspeed for the kite, in m/s\n      gs_position:\n        Array like position vector specifying the tether attach point in\n        ground coordinates. If not specified, will default to [0,0,0]\n      v_w_h_hub_cut_out:\n        Wind speed at virtual hub height for cut out.\n        Will truncate v_w range in KitePowerCurve if exceeded, and is\n        used to clean up power curve in KitePowerCurve object\n        and provide ref lines in plotting tools.\n      v_w_h_hub_cut_in:\n        Wind speed at virtual hub height for cut in.\n        Used to clean up power curve in KitePowerCurve object and\n        provide ref lines in plotting tools.\n      v_a_max:\n        maximum kite airspeed, in m/s\n      rotor_area:\n        Total rotor area (all rotors) in m^2.\n        Required if 'shaft_power_from_drag_power' does not provide\n        constraints, in which case local Betz limit is calculated and applied\n        as a simple constraint.\n        See 'shaft_power_from_drag_power' item for details.\n\n        Entirely ignored if 'shaft_power_from_drag_power' function in config\n        provides constraints, as it's assumed the rotor function is doing\n        its own, more detailed modeling.\n\n"

kpc_doc_dict = OrderedDict()

for line in six.ensure_str(config).split('\n'):
  if ':' in line:
    key = six.ensure_str(line).split(':')[0].lstrip()
    desc = ''
  else:
    desc += six.ensure_str(line.lstrip())+' '
    kpc_doc_dict[key] = desc

# for k, v in kpc_doc_dict.items():
#   print '{}: {}'.format(k,v)


# View available options and select the power curve to view

In [ ]:
#@title View available studies
available_studies = os.listdir(power_curve_parent_path)
print('Avaliable studies:')
pp(available_studies)


In [ ]:
#@title User-input: Manually set the target study folder

# study = '20190717_clean_stackup'
# study = '20190722_Fig8s_splines'
# study = '20190729_pencil_specs'
# study = '20190805_opt_sensitivities'
study = '20190829_test_c_lmn_limits'

print('Using the study:')
print(study)


In [ ]:
#@title View available power curves
available_pcs = sorted([pc[:-5] for pc in os.listdir(os.path.join(power_curve_parent_path, study)) if six.ensure_str(pc).endswith('.json')])
print('Avaliable power curves:')
pp(available_pcs)


In [ ]:
#@title User-input: Manually set the target power curve

# pc_name = 'targetHover_harmonic'
# pc_name = 'targetHover2_harmonic'
# pc_name = 'targetHoverSmallSymBridle8_fig8_8spline20pose_smallsteps_lowtol'
# pc_name = 'targetHoverSmallSymBridle8_fig8_12spline24pose_smallsteps_lowtol'
# pc_name = 'targetHover2_harmonicreexport'
# pc_name = available_pcs[1]
pc_name = 'BigM600_r02a_v04-circle-90m'
pc_name = 'BigM600_r02a_v05-circle-90m'
# pc_name = 'base_circle'
# pc_name = 'Ellipticalspec01r01clean_tightertol_18pos_6spline'
# pc_name = 'Ellipticalspec01r01clean_tighttol'

print('Viewing the power curve:')
print(pc_name)


# View the selected power curve

In [ ]:
#@title Load json
pc_params = json.loads(open(os.path.join(power_curve_parent_path, study, six.ensure_str(pc_name) + '.json')).read())

print('Loaded power curve:')
print('    ' + six.ensure_str(pc_name)) 

print() 
if 'pc_desc' in pc_params:
  print(pc_params['pc_desc'])
else:
  print('No power curve description available.')
if 'export_datetime' in pc_params:
  print('PowerCurve data exported at:')
  print(pc_params['export_datetime'])

print()
print('Available keys:')
pp(list(pc_params.keys()))

all_images = [f for f in os.listdir(os.path.join(power_curve_parent_path, study)) if (f.startswith(pc_name) and six.ensure_str(f).endswith('.png'))]
# print 
# print 'available images:'
# pp((all_images))

possible_images = [six.ensure_str(pc_name)+'_'+six.ensure_str(plot) for plot in plot_order]
# print 'preferred image order:'
# pp((possible_images))

images = [image for image in possible_images if image in all_images] + [image for image in all_images if image not in possible_images]
print()
print('Images to plot:')
pp((images))


In [ ]:
#@title Kite config
#@markdown note: method specifics cannot be stored in a json, so may be hidden from what is below.



"""config: a dictionary that specifies a kite configuration
      Required items:
        <aero_model>:
          An aero model for the kite (no tether) must be provided.
          options are:
            aero_coeff_from_alpha_beta:
              as above, but includes aero side lift coefficient, cY
            body_coeff_from_alpha_beta:
              as above, but body coefficients including cy.
              note that cY is aero coefficient, and cy is body.
        eta_shaft_to_pad:
          a function of (power_shaft) that returns a total efficiency for
          the system from shaft to padmount
        m_kite:
          mass of the kite in kg
        m_tether':
          mass of the tether in kg
        power_shaft_max:
          maximum net (all motors) shaft power limit, in watts
        s:
          kite reference wing area, in m^2
        l_tether:
          length of the tether from GS attach point to the wing
          (including bridle length)
        shaft_power_from_drag_power:
          a function of (rho, v_a, rotor_force) that returns a dictionary
          a negative force is generation

          Required items:
            power_shaft:
              shaft power, in W
          Optional items:
            anything desired to be appended to state, such as performance
            coefficients, status messages, additional constraints (see code
            for format)
            things appended to state are accessible by all plotting tools, etc

            Important notes:
              If constraints are provided it's assumed the function is doing its
              own stall and/or Betz checks. If none are provided, a Betz limit
              constraint is applied to the pose, and 'rotor_area' (total) is
              required to be in the config.
      Optional items:
        alpha_min:
          Minimum alpha, applied as a constraint.
        alpha_max:
          Maximum alpha, applied as a constraint.
        cL_max:
          Maximum coefficient of lift, applied as a constraint.
        cL_offset:
          Additional aero coefficient offset to be applied to the kite
        cY_offset:
          Additional lateral lift coefficient offset to be applied to the kite
        cD_offset:
          Additional drag coefficient offset to be applied to the kite
        cD_eff_tether:
          Effective drag coefficient of the tether at the kite, referenced to
          wing area.
        tension_max:
          Maximum operating tension, in Newtons. Applied as a constraint.
        tension_min:
          Min operating tension, in Newtons. Applied as a constraint.
        incl_max:
          Maximum inclination for any individual pose. Applied as a constraint.
        h_min:
          Minimum height AGL for any individual pose. Applied as a constraint.
        roll_min:
          Minimum roll angle, tether to kite.
          Positive roll is in to nominal clockwise looking downwind circle.
          Ie: rolling to kite's left.
        roll_max:
          Maximum roll angle, tether to kite.
          See roll_min for details.
        v_a_min:
          Minimum allowable airspeed for the kite, in m/s.
        v_a_max:
          Maximum allowable airspeed for the kite, in m/s
        gs_position:
          Array like position vector specifying the tether attach point in
          ground coordinates. If not specified, will default to [0,0,0]
        v_w_h_hub_cut_out:
          Wind speed at virtual hub height for cut out.
          Will truncate v_w range in KitePowerCurve if exceeded, and is
          used to clean up power curve in KitePowerCurve object
          and provide ref lines in plotting tools.
        v_w_h_hub_cut_in:
          Wind speed at virtual hub height for cut in.
          Used to clean up power curve in KitePowerCurve object and
          provide ref lines in plotting tools.
        v_a_max:
          maximum kite airspeed, in m/s
        rotor_area:
          Total rotor area (all rotors) in m^2.
          Required if 'shaft_power_from_drag_power' does not provide
          constraints, in which case local Betz limit is calculated and applied
          as a simple constraint.
          See 'shaft_power_from_drag_power' item for details.

          Entirely ignored if 'shaft_power_from_drag_power' function in config
          provides constraints, as it's assumed the rotor function is doing
          its own, more detailed modeling."""

from __future__ import division

from __future__ import absolute_import

from __future__ import print_function

import six


config = copy.deepcopy(pc_params['config'])
description = config.pop('description')
print(pc_name)
print()
print(description)
print()

for k, v in kpc_doc_dict.items():
  if k in config:
    print('{}: {}'.format(k, v))
    print('    {}'.format(config.pop(k)))
  # else:
  #   print 'no {}'.format(k)

print() 
print('remaining parameters:')
pp(config)


In [ ]:
#@title PowerCurve solve summary

print()
print((pc_params['solve_summary']))

print()
print('optimization settings:')
pp(pc_params['opt_params'])


In [ ]:
#@title Plots
#@markdown You can view this cell's outputs in fullscreen through the 3-dot menu. Also useful, right-click an image and 'Open image in new tab'.

for image in images:
  image_path = os.path.join(power_curve_parent_path, study, image)
  img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img)


In [ ]:
#@title More details

print('Wind resource:')
pp(pc_params['resource'])

if 'recent_stackup' in pc_params:
  print()
  print('recent commit stackup')
  # pp(pc_params['commit'])
  pp(pc_params['recent_stackup'])
else:
  print('no commit stackup available')


# View data

In [ ]:
#@title Explore available data

print('Main data available in "loop_params"')
pp(list(pc_params['loop_params'].keys()))
for k, v in pc_params['loop_params'].items():
  print('  keys in {}:'.format(k))
  pp(list(v[0].keys()))

print()
ii = 5
print('for example, looping at wind speed of {} m/s'.format(pc_params['wind_params']['v_ws_at_h_ref'][ii]))
pp(pc_params['loop_params']['pose_states_param'][ii])

print()
print('for all loops:')
pp(pc_params['loop_params']) 
